In [4]:
!nvidia-smi

Tue Dec 23 16:23:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# @title Step 1: Setup Environment (Corrected)
# We install from PyPI (stable) instead of GitHub to avoid version conflicts
!pip install -q transformers bitsandbytes accelerate pillow pandas

from google.colab import drive
drive.mount('/content/drive')

print("Environment ready & Drive mounted.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.4 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment ready & Drive mounted.


In [6]:
pip install -U bitsandbytes

In [7]:
# @title Fix: Clear Corrupted LLaVA Cache
import shutil
import os
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define the specific corrupted folder path
# This is the exact path where Hugging Face stores the LLaVA model
corrupted_path = "/content/drive/MyDrive/hf_models/models--llava-hf--llava-1.5-7b-hf"

# 3. Delete it
if os.path.exists(corrupted_path):
    print(f"🗑️ Found corrupted cache at: {corrupted_path}")
    print("Deleting it now... (this may take a minute)")
    shutil.rmtree(corrupted_path)
    print("✅ Deleted. You can now re-run Step 2.")
else:
    print("⚠️ Path not found. It might have already been deleted.")

# 4. Check Space (Crucial Step)
total, used, free = shutil.disk_usage("/content/drive")
free_gb = free // (2**30)
print(f"\nFree Space in Drive: {free_gb} GB")

if free_gb < 6:
    print("⚠️ WARNING: You have less than 6GB free!")
    print("❌ The download will fail again unless you free up space.")
    print("👉 Please go to Google Drive and empty your Trash.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🗑️ Found corrupted cache at: /content/drive/MyDrive/hf_models/models--llava-hf--llava-1.5-7b-hf
Deleting it now... (this may take a minute)
✅ Deleted. You can now re-run Step 2.

Free Space in Drive: 70 GB


In [1]:
# @title Step 2: Load Model (Saved to Google Drive)
import torch
import os
from google.colab import drive
from transformers import BitsAndBytesConfig, AutoProcessor, LlavaForConditionalGeneration

# 1. Mount Google Drive to save the model permanently
# drive.mount('/content/drive')

# 2. Define the cache folder in your Drive
# This creates a folder named 'hf_models' inside your Drive to store the files
drive_cache_dir = "/content/drive/MyDrive/hf_models"
os.makedirs(drive_cache_dir, exist_ok=True)
print(f"✅ Model will be cached at: {drive_cache_dir}")

# 3. Configure Quantization (Same as before)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Old model
model_id = "llava-hf/llava-1.5-7b-hf"


print(f"Loading {model_id}... (checking Drive first)")

# 4. Load Processor with Cache
processor = AutoProcessor.from_pretrained(
    model_id,
    cache_dir=drive_cache_dir  # <--- This tells it to look in Drive
)

# 5. Load Model with Cache
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    cache_dir=drive_cache_dir  # <--- This tells it to look in Drive
)

print("Model loaded successfully!")

✅ Model will be cached at: /content/drive/MyDrive/hf_models
Loading llava-hf/llava-1.5-7b-hf... (checking Drive first)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Model loaded successfully!


In [16]:
# @title Step 3: Define "Combined Strategy" Pipeline (Updated for LoRA Dataset Structure)
from PIL import Image
import torch
import pandas as pd
from tqdm.notebook import tqdm
import os
import shutil  # Added for copying images

def run_llava(image, text_prompt, max_tokens=500):
    full_prompt = f"USER: <image>\n{text_prompt}\nASSISTANT:"
    inputs = processor(text=full_prompt, images=image, return_tensors="pt").to("cuda")
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        repetition_penalty=1.2
    )
    generated_text = processor.decode(output_ids[0], skip_special_tokens=True)
    if "ASSISTANT:" in generated_text:
        return generated_text.split("ASSISTANT:")[-1].strip()
    return generated_text

def get_detailed_annotation(image, context_info):
    # --- FIX: FULL 13-ITEM EXAMPLES FOR CONSISTENCY ---
    prompt = f"""Act as a data extraction engine.

CONTEXT FACTS:
{context_info}

INSTRUCTIONS:
- Return ONLY the numbered list (1-13).
- Use SHORT phrases (max 5-6 words per line).
- NO filler words. NO full sentences.

EXAMPLE 1 (Required Format):
1. Skin color and gender: White male, fair.
2. Overall facial appearance: Middle-aged, approx 45.
3. Hair: Blonde, short, receding hairline.
4. Forehead: High, smooth.
5. Hairline: Receding, M-shape.
6. Beard: Goatee, reddish-brown.
7. Eyes: Blue, oval, no glasses.
8. Eyebrows: Brown, thick.
9. Nose: Straight, average width.
10. Lips: Thin, pink.
11. Chin and jawline: Square jaw, dimpled chin.
12. Ears: Visible, attached lobes.
13. Scars: None.

EXAMPLE 2 (Required Format):
1. Skin color and gender: Black female, dark skin.
2. Overall facial appearance: Young adult, approx 25.
3. Hair: Long, black, braided.
4. Forehead: Average height, smooth.
5. Hairline: Straight, low.
6. Beard: None.
7. Eyes: Brown, large, almond.
8. Eyebrows: Black, arched, thin.
9. Nose: Button shape, wide.
10. Lips: Full, dark liner.
11. Chin and jawline: Pointed chin, smooth jaw.
12. Ears: Hidden by hair.
13. Scars: Small scar on left cheek.

REAL TASK (Extract data for the image below):
1. Skin color and gender:
2. Overall facial appearance (estimated age):
3. Hair (color, length, texture):
4. Forehead (shape, features):
5. Hairline (shape, receding):
6. Beard (If none, write "None"):
7. Eyes (color, shape, glasses):
8. Eyebrows (color, thickness):
9. Nose (shape, width):
10. Lips (size, thickness):
11. Chin and jawline (shape):
12. Ears (visible?):
13. Scars (location):"""
    return run_llava(image, prompt, max_tokens=400)

def compress_for_sd(detailed_text, real_image):
    # --- FIX: 100 WORD LIMIT + MANDATORY ATTRIBUTES ---
    prompt = f"""You are a captioning assistant. Transform the description into a single dense training prompt for SDXL.

INSTRUCTIONS:
- You MUST include: Race, Gender, Age, Hair (Color/Style), Beard (if any), and distinct facial features.
- Do NOT use conversational filler ("The subject is", "In the picture").
- Keep it under 100 words.
- Focus on visual physical traits.

EXAMPLE INPUT:
1. Skin: White male. 2. Hair: Blonde, short. 3. Beard: Goatee. 4. Eyes: Blue.
EXAMPLE OUTPUT:
A middle-aged white male with short blonde hair, blue eyes, and a reddish goatee. He has a fair complexion, a high forehead, and a square jawline.

REAL TASK:
INPUT DESCRIPTION:
{detailed_text}

OUTPUT:"""
    return run_llava(real_image, prompt, max_tokens=500)

def process_batch(folder_path, csv_path):
    results = []

    # 1. Setup LoRA Dataset Folders
    # Identify parent directory
    parent_dir = os.path.dirname(folder_path.rstrip('/'))

    # Define LoRA paths
    lora_root = os.path.join(parent_dir, "lora_dataset")
    captions_dir = os.path.join(lora_root, "captions")
    images_dir = os.path.join(lora_root, "images")

    # Create directories if they don't exist
    os.makedirs(captions_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)

    print(f"📂 LoRA Dataset structure created at: {lora_root}")
    print(f"   ├── 📂 {captions_dir}")
    print(f"   └── 📂 {images_dir}")

    # 2. Load CSV
    if not os.path.exists(csv_path):
        print(f"❌ Error: CSV file not found at {csv_path}")
        return pd.DataFrame()

    metadata_df = pd.read_csv(csv_path)
    metadata_df['inmate_id'] = metadata_df['inmate_id'].astype(str)
    print(f"✅ Loaded Metadata for {len(metadata_df)} inmates.")

    # 3. Get Images
    valid_extensions = ('.jpg', '.jpeg', '.png', '.webp')
    if not os.path.exists(folder_path):
        print(f"Error: Folder {folder_path} does not exist.")
        return pd.DataFrame()

    all_files = [f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)]

    # --- TESTING LIMIT (10 Images) ---
    image_files = all_files[:10]
    print(f"Processing first {len(image_files)} images (Test Mode)...")
    # ----------------------------------

    # 4. Process Loop
    for filename in tqdm(image_files, desc="Processing Images"):
        img_path = os.path.join(folder_path, filename)
        file_id = os.path.splitext(filename)[0]

        # Match CSV Data
        row = metadata_df[metadata_df['inmate_id'] == file_id]
        context_str = "No specific data available."
        if not row.empty:
            age = row.iloc[0]['age']
            race = row.iloc[0]['race']
            sex = row.iloc[0]['sex']
            context_str = f"Subject is a {age} year old {race} {sex}."

        try:
            image = Image.open(img_path).convert("RGB")

            # Phase 1: Annotation
            annotation = get_detailed_annotation(image, context_str)

            # Phase 2: Compression
            final_prompt = compress_for_sd(annotation, image)

            # --- SAVE OPERATIONS ---

            # A. Save Caption (.txt)
            txt_filename = f"{file_id}.txt"
            txt_path = os.path.join(captions_dir, txt_filename)
            with open(txt_path, "w") as f:
                f.write(final_prompt)

            # B. Copy Image (.jpg/.png)
            target_img_path = os.path.join(images_dir, filename)
            shutil.copy(img_path, target_img_path)

            results.append({
                "Filename": filename,
                "Detailed_Annotation": annotation,
                "Final_Prompt": final_prompt
            })

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    return pd.DataFrame(results)

In [17]:
# @title Step 4: Run Batch & Save Results
import os

# --- PATHS ---
folder_path = "/content/drive/MyDrive/project_dataset/output"
csv_path = "/content/drive/MyDrive/project_dataset/df_final.csv"
# -------------

if os.path.exists(folder_path):
    # Run the batch processing
    df_results = process_batch(folder_path, csv_path)

    # Display results
    from IPython.display import display
    print("\nProcessing Complete! Here are the results:")
    display(df_results.head())

    # Save summary CSV in the PARENT folder
    parent_dir = os.path.dirname(folder_path.rstrip('/'))
    save_path = os.path.join(parent_dir, "Final_results_merged.csv")

    df_results.to_csv(save_path, index=False)
    print(f"\n✅ Summary CSV saved to: {save_path}")
    print(f"✅ Images & Captions ready in: {os.path.join(parent_dir, 'lora_dataset')}")

else:
    print(f"Error: Folder not found at {folder_path}")

📂 LoRA Dataset structure created at: /content/drive/MyDrive/project_dataset/lora_dataset
   ├── 📂 /content/drive/MyDrive/project_dataset/lora_dataset/captions
   └── 📂 /content/drive/MyDrive/project_dataset/lora_dataset/images
✅ Loaded Metadata for 10000 inmates.
Processing first 10 images (Test Mode)...


Processing Images:   0%|          | 0/10 [00:00<?, ?it/s]


Processing Complete! Here are the results:


,Filename,Detailed_Annotation,Final_Prompt
0,R85534.jpg,"1. Skin color and gender: White female, light ...","A young woman with long blonde hair, blue eyes..."
1,Y11692.jpg,1. Skin color and gender: Dark skinned woman.\...,A dark-skinned female late teenager or young a...
2,R87791.jpg,"1. Skin color and gender: White female, light ...","A young woman wearing a white shirt, dark brow..."
3,Y28958.jpg,"1. Skin color and gender: White female, light ...",A young woman wearing a white shirt smiles at ...
4,Y26307.jpg,1. Skin color and gender: Dark skinned woman.\...,A dark skinned female late teenager or young a...



✅ Summary CSV saved to: /content/drive/MyDrive/project_dataset/Final_results_merged.csv
✅ Images & Captions ready in: /content/drive/MyDrive/project_dataset/lora_dataset


In [18]:
# @title Step 6: Save Results to Word Document (.docx)
# 1. Install the library
!pip install -q python-docx

import os
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH

# 2. Setup Paths
# We need to find the CSV in the PARENT directory, not inside the images folder
parent_dir = os.path.dirname(folder_path.rstrip('/'))
csv_path = os.path.join(parent_dir, "Final_results_merged.csv")
word_output_path = os.path.join(parent_dir, "VLM_Pipeline_Results_Final.docx")

print(f"📂 Looking for CSV at: {csv_path}")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"✅ Loaded data for {len(df)} entries.")

    # 3. Create the Word Document
    doc = Document()

    # Add a Title
    title = doc.add_heading('VLM Pipeline Results Report', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    doc.add_paragraph(f"Total Images Processed: {len(df)}")
    doc.add_paragraph("-" * 50)

    # 4. Iterate and Add Content
    for idx, row in df.iterrows():
        img_filename = row['Filename']
        # The images are still inside the 'folder_path' (output folder)
        img_path = os.path.join(folder_path, img_filename)

        # --- Header ---
        header = doc.add_heading(f"Image: {img_filename}", level=1)

        # --- Image ---
        if os.path.exists(img_path):
            try:
                # Add image and resize to 3 inches width (fits nicely on page)
                doc.add_picture(img_path, width=Inches(3.0))
                last_paragraph = doc.paragraphs[-1]
                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            except Exception as e:
                doc.add_paragraph(f"[Error loading image: {e}]")
        else:
            doc.add_paragraph("[Image file not found]")

        # --- Phase 1: Annotation ---
        doc.add_heading('Phase 1: Detailed Annotation', level=2)
        # Handle cases where data might be missing/NaN
        anno_text = str(row.get('Detailed_Annotation', 'N/A'))
        p_anno = doc.add_paragraph(anno_text)
        p_anno.style = 'Quote' # Makes it look distinct

        # --- Phase 2: Compressed Prompt ---
        doc.add_heading('Phase 2: Compressed Prompt (SDXL)', level=2)
        prompt_text = str(row.get('Final_Prompt', 'N/A'))
        p_prompt = doc.add_paragraph(prompt_text)

        # Calculate Word Count
        word_count = len(prompt_text.split())

        # Add Stats
        stats = doc.add_paragraph()
        run = stats.add_run(f"Word Count: {word_count} words")
        run.bold = True
        run.font.color.rgb = RGBColor(0, 100, 0) # Dark Green

        # --- Separator ---
        doc.add_page_break()
        # Optional: Print progress every 10 items
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1} entries...")

    # 5. Save the Document
    doc.save(word_output_path)
    print(f"\n🎉 Success! Word Document saved at:\n{word_output_path}")

else:
    print(f"❌ Error: Could not find results CSV at {csv_path}")
    print("Please make sure you ran Step 4 successfully.")

📂 Looking for CSV at: /content/drive/MyDrive/project_dataset/Final_results_merged.csv
✅ Loaded data for 10 entries.
Processed 10 entries...

🎉 Success! Word Document saved at:
/content/drive/MyDrive/project_dataset/VLM_Pipeline_Results_Final.docx
